# Data integration

For each sub-dataset, write (and execute) code that converts a file (using possibly an old schema) into a file that has the new, latest schema version.

Your conversion code should not modify the original files, but instead create a new file. 2

Be sure to explain the design behind your conversion functions!

The data integration step is highly parallellizable. Therefore, your solution on this part
**must** be written in Spark

$\color{red}{\text{ASSUMPTIONS!!!.}}$ 
<br>
$\color{red}{\text{You must run the t1_explore notebook before running this notebook. }}$ 
<br>
$\color{red}{\text{The taxi zone folder must be in thesame location as this notebook. }}$ 

In [22]:
from pyspark.sql import SparkSession

#if a spark session was already started, we stop it before starting a new one
#(there can be only one spark context per jupyter notebook)
try: 
    spark
    print("Spark application already started. Terminating existing application and starting new one")
    spark.stop()
except: 
    pass

# Create a new spark session (note, the * indicates to use all available CPU cores)
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("TLC") \
    .getOrCreate()
    
sc=spark.sparkContext

Spark application already started. Terminating existing application and starting new one


In [23]:
# defining some key functions
#generator function
def file_content(main_folder):
    file_list = os.listdir(main_folder)
    for file_name in file_list:
        yield file_name

In [24]:
from pyspark.sql import *
from pyspark.sql.functions import *
# Function to fix column names

def column_name_fix(df):
    '''Function converts column names to lowercase and remove whitespaces'''
    colnames = df.columns
    for x in range(len(colnames)):
        colnames[x] =  colnames[x].strip()
        colnames[x] =  colnames[x].lower()
        
    df = df.toDF(*colnames)
    return df

In [25]:
# Function to drop column
def column_drop(df,drop_list):
    '''Function receives a spark dataframe and list of columns to be dropped.
    It returns a dataframe less the columns specified to be dropped'''
    y = []
    for x in drop_list:
        y.append(df.columns[x])
    return df.drop(*y)

In [26]:
# Function to rename column
def column_rename(df,col_list1,col_list2):
    '''Function renames the column name of a dataframe with a provided list of column names.
        The two lists must be of the order i.e. value one in list one replaces value one in list two and so on'''
    if len(col_list1) == len(col_list2):
        for x in range(len(col_list1)):
            df = df.select('*', df[col_list1[x]].alias(col_list2[x]))
        df = column_drop(df,col_list1)
        return df
    else:
        print('length of two list must be thesame.')
        pass

In [27]:
# Add columns to a dataframe
from pyspark.sql.functions import lit
def column_add(df,col_list):
    
    for x in col_list:
        df = df.withColumn(x, lit(""))
    return df


Loading the shapefile using geopandas (note: GeoPandas is not installed by default. If you use anaconda, you can install it by simply running conda install geopandas. 

In [28]:
pip install geopandas

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [29]:
import matplotlib.pyplot as plt 
import geopandas as gpd
from shapely.geometry import Point, Polygon

In [30]:
# Load the shapefile, this yields a GeoDataFrame that has a row for each zone
zones = gpd.read_file('./taxi_zones/taxi_zones.shp')

In [31]:
# Now re-project the coordinates to the CRS EPSG 4326, which is the CRS used in GPS (https://epsg.io/4326)
zones = zones.to_crs({'init':'epsg:4326'})

In [32]:
pip install pygeos

  Using cached https://files.pythonhosted.org/packages/8c/45/7dff89c1af72ed5f479a611c353768ef8309bfc1a9f5fe670ccb4be97872/pygeos-0.8-cp37-cp37m-manylinux1_x86_64.whl
ERROR: Could not install packages due to an EnvironmentError: [Errno 13] Permission denied: '/usr/local/anaconda3/lib/python3.7/site-packages/pygeos.libs'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [33]:
# create an R-tree index on it's geometry

rtree = zones.sindex

In [34]:
def location_id(df1,zones):
    n = len(df1)
    for i in range(n):
        query_point = Point( float(0 if (df1.iloc[i].pickup_longitude) is None else (df1.iloc[i].pickup_longitude)), float(0 if (df1.iloc[i].pickup_latitude) is None else (df1.iloc[i].pickup_latitude)))
        
        possible_matches = list(rtree.intersection( query_point.bounds ))
       
        for x in possible_matches:
            if zones.iloc[x].geometry.contains(query_point)==True:
                df1.pulocationid[i] = zones.iloc[x].LocationID
        
        query_point2 = Point( float(0 if (df1.iloc[i].dropoff_longitude) is None else (df1.iloc[i].dropoff_longitude)), float(0 if (df1.iloc[i].dropoff_latitude) is None else (df1.iloc[i].dropoff_latitude)))
        possible_matches = list(rtree.intersection( query_point2.bounds ))
        for x in possible_matches:
            if zones.iloc[x].geometry.contains(query_point2)==True:
                df1.dolocationid[i] = zones.iloc[x].LocationID
    
    return df1   

In [35]:
# Creating a folder to hold integrated files
# Check wether folder exist if not create
import os
if os.path.exists('Files/integrated_files'):
    pass
else:
    os.mkdir('Files/integrated_files')

### Integrating FHV files

In [36]:
#FHV taxi files

# Check wether folder exist if not create
if os.path.exists('Files/integrated_files/FHV'):
    pass
else:
    os.mkdir('Files/integrated_files/FHV')

# Schema One

import os
if os.path.exists('Files/integrated_files/FHV/Schema_v_1'):
    pass
else:
    os.mkdir('Files/integrated_files/FHV/Schema_v_1')
# Columns to be renamed
col_list1 = [1]
col_list2 = ['pickup_datetime']

# columns to be added
col_add = ['dropoff_datetime', 'pulocationid', 'dolocationid', 'sr_flag', 'dispatching_base_number']

# columnns to be dropped
col_drop = [2]

folder_path = './Files/FHV/v_1'
for file in file_content(folder_path):
        file_path = os.path.join('Files/FHV/v_1', file)
        df = column_name_fix(spark.read.csv(file_path, header=True))# The read process pass via column_name_fix function
        df = column_drop(df, col_drop)
        df = column_rename(df,col_list1,col_list2)
        df = column_add(df,col_add)
        df.toPandas().to_csv(os.path.join('Files/integrated_files/FHV/Schema_v_1', file),index=False)
        
     

In [37]:
#FHV taxi files
# Schema Two

import os
if os.path.exists('Files/integrated_files/FHV/Schema_v_2'):
    pass
else:
    os.mkdir('Files/integrated_files/FHV/Schema_v_2')
    
# Columns to be renamed
col_list1 = []
col_list2 = []

# columns to be added
col_add = ['sr_flag', 'dispatching_base_number']

# columnns to be dropped
col_drop = []

folder_path = './Files/FHV/v_2'
for file in file_content(folder_path):
        file_path = os.path.join('Files/FHV/v_2', file)
        df = column_name_fix(spark.read.csv(file_path, header=True))# The read process pass via column_name_fix function
        df = column_drop(df, col_drop)
        df = column_rename(df,col_list1,col_list2)
        df = column_add(df,col_add)
        df.toPandas().to_csv(os.path.join('Files/integrated_files/FHV/Schema_v_2', file),index=False)


In [38]:
#FHV taxi files
# Schema Three

import os
if os.path.exists('Files/integrated_files/FHV/Schema_v_3'):
    pass
else:
    os.mkdir('Files/integrated_files/FHV/Schema_v_3')
    
# Columns to be renamed
col_list1 = []
col_list2 = []

# columns to be added
col_add = ['dispatching_base_number']

# columnns to be dropped
col_drop = []

folder_path = './Files/FHV/v_3'
for file in file_content(folder_path):
        file_path = os.path.join('Files/FHV/v_3', file)
        df = column_name_fix(spark.read.csv(file_path, header=True))# The read process pass via column_name_fix function
        df = column_drop(df, col_drop)
        df = column_rename(df,col_list1,col_list2)
        df = column_add(df,col_add)
        df.toPandas().to_csv(os.path.join('Files/integrated_files/FHV/Schema_v_3', file),index=False)
 

In [39]:
# FHV taxi files 
# Schema Four
import shutil
if os.path.exists('Files/integrated_files/FHV/Schema_v_4'):
    pass
else:
    os.mkdir('Files/integrated_files/FHV/Schema_v_4')
folder_path = './Files/FHV/v_4/'    
# moving the files to 
for file in os.listdir(folder_path):
    #shutil.copy2(os.path.join('Files/FHV/v_4', file), 'Files/integrated_files/FHV/Schema_v_4/')
    my_file = folder_path + file
    shutil.copyfile(my_file,'Files/integrated_files/FHV/Schema_v_4/'+file)

### Integrating FHVHV

In [40]:
# Check wether folder exist if not create
if os.path.exists('Files/integrated_files/FHVHV'):
    pass
else:
    os.mkdir('Files/integrated_files/FHVHV')

import shutil
if os.path.exists('Files/integrated_files/FHVHV/Schema_v_1'):
    pass
else:
    os.mkdir('Files/integrated_files/FHVHV/Schema_v_1')
    
# moving the files to 
for file in os.listdir('./Files/FHVHV/v_1'):
    shutil.copy2(os.path.join('Files/FHVHV/v_1', file), 'Files/integrated_files/FHVHV/Schema_v_1')

### Integrating green files

In [41]:
# green taxi files

# Check wether folder exist if not create
if os.path.exists('Files/integrated_files/green'):
    pass
else:
    os.mkdir('Files/integrated_files/green')


# schema one
import os
if os.path.exists('Files/integrated_files/green/Schema_v_1'):
    pass
else:
    os.mkdir('Files/integrated_files/green/Schema_v_1')
    
# Columns to be renamed
col_list1 = []
col_list2 = []

# columns to be added
col_add = ['pulocationid', 'dolocationid','improvement_surcharge', 'congestion_surcharge']

# columnns to be dropped
col_drop = ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']

folder_path = './Files/green/v_1'
for file in file_content(folder_path):
        file_path = os.path.join('Files/green/v_1', file)
        df = column_name_fix(spark.read.csv(file_path, header=True))# The read process pass via column_name_fix function
        df = column_add(df,col_add)
        df = df.toPandas()
        df = location_id(df,zones)
        df = df.drop(columns= col_drop)
        df.to_csv(os.path.join('Files/integrated_files/green/Schema_v_1', file),index=False)

In [42]:
# green taxi files
# schema Two

import os
if os.path.exists('Files/integrated_files/green/Schema_v_2'):
    pass
else:
    os.mkdir('Files/integrated_files/green/Schema_v_2')
    
# Columns to be renamed
col_list1 = []
col_list2 = []

# columns to be added
col_add = ['pulocationid', 'dolocationid', 'congestion_surcharge']
col_drop = ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']

folder_path = './Files/green/v_2'
for file in file_content(folder_path):
        file_path = os.path.join('Files/green/v_2', file)
        df = column_name_fix(spark.read.csv(file_path, header=True))# The read process pass via column_name_fix function
        df = column_add(df,col_add)
        df = df.toPandas()
        df = location_id(df,zones)
        df = df.drop(columns= col_drop)
        df.to_csv(os.path.join('Files/integrated_files/green/Schema_v_2', file),index=False)

In [43]:
# green taxi files
# schema Three

import os
if os.path.exists('Files/integrated_files/green/Schema_v_3'):
    pass
else:
    os.mkdir('Files/integrated_files/green/Schema_v_3')
    
# Columns to be renamed
col_list1 = []
col_list2 = []

# columns to be added
col_add = ['congestion_surcharge']
col_drop = ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']

folder_path = './Files/green/v_3'
for file in file_content(folder_path):
        file_path = os.path.join('Files/green/v_3', file)
        df = column_name_fix(spark.read.csv(file_path, header=True))# The read process pass via column_name_fix function
        df = column_add(df,col_add)
        df.toPandas().to_csv(os.path.join('Files/integrated_files/green/Schema_v_3', file),index=False)

In [44]:
# Schema Four
import shutil
if os.path.exists('Files/integrated_files/green/Schema_v_4'):
    pass
else:
    os.mkdir('Files/integrated_files/green/Schema_v_4')
    
# moving the files to 
for file in os.listdir('./Files/green/v_4'):
    shutil.copy2(os.path.join('Files/green/v_4', file), 'Files/integrated_files/green/Schema_v_4')

### Integrating yellow files

In [45]:
# yellow taxi files

# Check wether folder exist if not create
if os.path.exists('Files/integrated_files/yellow'):
    pass
else:
    os.mkdir('Files/integrated_files/yellow')


# schema one
import os
if os.path.exists('Files/integrated_files/yellow/Schema_v_1'):
    pass
else:
    os.mkdir('Files/integrated_files/yellow/Schema_v_1')
    
# Columns to be renamed
col_list1 = [0,1,2,5,6,7,8,9, 10, 13]
col_list2 = ['vendorid','tpep_pickup_datetime','tpep_dropoff_datetime','pickup_longitude', 'pickup_latitude','ratecodeid', 'store_and_fwd_flag', 'dropoff_longitude', 'dropoff_latitude','improvement_surcharge']

# columns to be added
col_add = ['pulocationid', 'dolocationid', 'congestion_surcharge']

# columnns to be dropped
col_drop = ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']

folder_path = './Files/yellow/v_1'
for file in file_content(folder_path):
        file_path = os.path.join('Files/yellow/v_1', file)
        df = column_name_fix(spark.read.csv(file_path, header=True))# The read process pass via column_name_fix function
        df = column_rename(df,col_list1,col_list2)
        df = column_add(df,col_add)
        df = df.toPandas()
        df = location_id(df,zones)
        df = df.drop(columns= col_drop)
        df.to_csv(os.path.join('Files/integrated_files/yellow/Schema_v_1', file),index=False)

In [46]:
# schema Two

import os
if os.path.exists('Files/integrated_files/yellow/Schema_v_2'):
    pass
else:
    os.mkdir('Files/integrated_files/yellow/Schema_v_2')
    
# Columns to be renamed
col_list1 = [0,1,2,7,8, 13]
col_list2 = ['vendorid','tpep_pickup_datetime','tpep_dropoff_datetime','ratecodeid','store_and_fwd_flag','improvement_surcharge']

# columns to be added
col_add = ['pulocationid', 'dolocationid', 'congestion_surcharge']

# columnns to be dropped
col_drop = ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']

# Redefining the schema
DDLSchema = DDLSchema = "vendor_id String, pickup_datetime String, dropoff_datetime String,passenger_count String, trip_distance String, pickup_longitude float, pickup_latitude float, rate_code String, store_and_fwd_flag String, dropoff_longitude float, dropoff_latitude float, payment_type String, fare_amount String, surcharge String, mta_tax String, tip_amount String, tolls_amount String, total_amount String"

folder_path = './Files/yellow/v_2'
for file in file_content(folder_path):
        file_path = os.path.join('Files/yellow/v_2', file)
        df = column_name_fix(spark.read.csv(file_path,header=True))# The read process pass via column_name_fix function
        df = column_rename(df,col_list1,col_list2)
        df = column_add(df,col_add)
        df = df.toPandas()
        df = location_id(df,zones)
        df = df.drop(columns= col_drop)
        df.to_csv(os.path.join('Files/integrated_files/yellow/Schema_v_2', file),index=False)


In [47]:
# schema Three

import os
if os.path.exists('Files/integrated_files/yellow/Schema_v_3'):
    pass
else:
    os.mkdir('Files/integrated_files/yellow/Schema_v_3')
    
# Columns to be renamed
col_list1 = []
col_list2 = []

# columns to be added
col_add = ['pulocationid', 'dolocationid', 'congestion_surcharge']

# columnns to be dropped
col_drop = ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']

folder_path = './Files/yellow/v_3'
for file in file_content(folder_path):
        file_path = os.path.join('Files/yellow/v_3', file)
        df = column_name_fix(spark.read.csv(file_path, header=True))# The read process pass via column_name_fix function   
        df = column_add(df,col_add)
        df = df.toPandas()
        df = location_id(df,zones)
        df = df.drop(columns= col_drop)
        df.to_csv(os.path.join('Files/integrated_files/yellow/Schema_v_3', file),index=False)

In [48]:
# schema Four

import os
if os.path.exists('Files/integrated_files/yellow/Schema_v_4'):
    pass
else:
    os.mkdir('Files/integrated_files/yellow/Schema_v_4')
    
# Columns to be renamed
col_list1 = []
col_list2 = []

# columns to be added
col_add = [ 'congestion_surcharge']

# columnns to be dropped
col_drop = []

folder_path = './Files/yellow/v_4'
for file in file_content(folder_path):
        file_path = os.path.join('Files/yellow/v_4', file)
        df = column_name_fix(spark.read.csv(file_path, header=True))# The read process pass via column_name_fix function   
        df = column_add(df,col_add)
        
        df.toPandas().to_csv(os.path.join('Files/integrated_files/yellow/Schema_v_4', file),index=False)


In [49]:
# Schema Five
import shutil
if os.path.exists('Files/integrated_files/yellow/Schema_v_5'):
    pass
else:
    os.mkdir('Files/integrated_files/yellow/Schema_v_5')
    
# moving the files to 
for file in os.listdir('./Files/yellow/v_5'):
    shutil.copy2(os.path.join('Files/yellow/v_5', file), 'Files/integrated_files/yellow/Schema_v_5')